In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from FAA import Ohare
from FAA import Hanger

In [2]:
import pandas as pd
faa = Ohare.Ohare()

In [3]:
faa.flights_from_ohare.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
10,2018,1,1,27,6,2018-01-27,UA,19977,UA,N39461,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2018,1,1,27,6,2018-01-27,UA,19977,UA,N37471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2018,1,1,27,6,2018-01-27,UA,19977,UA,N482UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2018,1,1,27,6,2018-01-27,UA,19977,UA,N57862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2018,1,1,27,6,2018-01-27,UA,19977,UA,N66808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
schema = pd.read_csv('../data/ohare/column_definitions.csv')
schema

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109


In [7]:
from sqlalchemy import create_engine

username = 'bjg'

params = { 'host': 'localhost',
           'port': 5432
}

connection_string = f'postgres://{username}:{params["host"]}@{params["host"]}:{params["port"]}/faa'

In [8]:
engine = create_engine(connection_string)

In [9]:
schema.to_sql('ORD_FLIGHTS', engine, index=False)

In [10]:
head = faa.flights_from_ohare.head()

In [11]:
head.to_sql('HEAD_TST', engine, index=False)

To prime the pump, I will need to read in one month of data to build the schema, because just reading in the column names causes the schema to be all text. So the process is looking like

* Read in one months data
* Get into Pandas Data Frame
* Connect to DataBase
* Write DF to database
* Write script to append remaining months to database.
* Dont forget to filter out ORD with grep before feeding month's in